In [8]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"],
# })
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
# np.set_printoptions(linewidth=300, formatter={'float': '{:.3e'.format})

<module 'Modules.figures' from 'C:\\Users\\gabri\\PycharmProjects\\tunable-capacitive-coupling-flux-qubits\\Modules\\figures.py'>

In [9]:
n_eig =5

In [10]:
fF  = 1e-15
nH  = 1e-9
GHz = 1e9

In [11]:
nmax_r = 20
nmax_f = 25

only_inner = True
compensate_extra_cap = False

In [12]:
CR_1, CF_1, LF_1, LR_1, EJ_1, Δ_1, ω_r_1 = sq_ext.get_experimental_parameters('qubit_1')
CR_2, CF_2, LF_2, LR_2, EJ_2, Δ_2, ω_r_2 = sq_ext.get_experimental_parameters('qubit_2')
CR_3, CF_3, LF_3, LR_3, EJ_3, Δ_3, ω_r_3 = sq_ext.get_experimental_parameters('qubit_3')

## Single qubit

In [13]:
## Testings
Δ=1
fluxonium = sq_ext.sq_fluxonium(C_F_eff=CF_1, L_F_eff=LF_1, EJ=EJ_1, Δ=Δ_1, nmax_f=nmax_f)
resonator = sq_ext.sq_resonator(C_R_eff=CR_1, L_R_eff=LR_1, Δ=Δ_1, nmax_f=nmax_f)

H_0 = sq_ext.hamiltonian_qubit(fluxonium = fluxonium, resonator = resonator, Δ=0) 
H   = sq_ext.hamiltonian_qubit(fluxonium = fluxonium, resonator = resonator, Δ=Δ) 

H1 = fluxonium.hamiltonian()
H2 = resonator.hamiltonian()

ψ_0_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2, solver='numpy', real=True)[1][:,0])
ψ_1_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2, solver='numpy', real=True)[1][:,1])
ψ_0_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2, solver='numpy', real=True)[1][:,0])
ψ_1_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2, solver='numpy', real=True)[1][:,1])

ψ_0 = [qt.tensor(ψ_0_f, ψ_0_r), qt.tensor(ψ_1_f,ψ_0_r), qt.tensor(ψ_0_f,ψ_1_r),  qt.tensor(ψ_1_f,ψ_1_r)]

In [14]:
H_eff_SWT = sq_ext.H_eff_SWT(H_0, H, n_eig=5)
Nf, Nr = sq_ext.sq_get_energy_indices_hamiltonian(H_qubit = H_0, H_fluxonium=H1, H_resonator=H2, n_eig=5)
subspace_indices = np.intersect1d(np.where(Nf<=1), np.where(Nr<=1)).tolist()

H_eff_SWT_subspace  = H_eff_SWT [subspace_indices][:, subspace_indices]
P = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_subspace, print_pretty=True)

 9.871143050325104	*	 I ⨂I 
 -1.8340398410991685	*	 I ⨂σz 
 0.00022102509063515186	*	 σx ⨂σx 
 0.20461063896483334	*	 σy ⨂σy 
 -3.2187759908174325	*	 σz ⨂I 
 -0.00708212792486651	*	 σz ⨂σz 


In [15]:
# subspace_indices = [0,1,2,4]

In [16]:
E_0  = [ (ψ_0_i.dag() * H_0 * ψ_0_i).data[0,0]/2/np.pi/GHz for ψ_0_i in ψ_0]
E, ψ = sq_ext.diag(H  , n_eig, out='GHz', solver='Qutip', qObj=True)
subspace_indices = sq_ext.find_close_indices(E_0,E,tol=0.05)
ψ = ψ[subspace_indices]
E = E[subspace_indices]
H_eff_SWT_large = sq_ext.H_eff_SWT_large(ψ_0, ψ, E)
P_large = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_large, print_pretty=True)

 9.879611414170446	*	 I ⨂I 
 -1.8425084880250213	*	 I ⨂σz 
 0.20482085126469565	*	 σx ⨂σx 
 1.0812790769837433e-05	*	 σy ⨂σy 
 -3.227244637743288	*	 σz ⨂I 
 0.0013862359205289998	*	 σz ⨂σz 


In [17]:
H_eff_p1_subspace  = sq_ext.H_eff_p1(H_0, H, n_eig) [subspace_indices][:, subspace_indices]
sq_ext.decomposition_in_pauli_4x4(H_eff_p1_subspace, print_pretty=True);

 9.885596503352916	*	 I ⨂I 
 -1.8443489410367235	*	 I ⨂σz 
 0.20468677428949716	*	 σx ⨂σx 
 -3.2299999999997944	*	 σz ⨂I 


In [18]:
H_eff_p1_subspace  = sq_ext.H_eff_p1_large(ψ_0, H)
sq_ext.decomposition_in_pauli_4x4(H_eff_p1_subspace, print_pretty=True);

 9.8855965033529	*	 I ⨂I 
 -1.8443489410367129	*	 I ⨂σz 
 0.20468677428949691	*	 σx ⨂σx 
 -3.2299999999997833	*	 σz ⨂I 


C:\Users\gabri\PycharmProjects\tunable-capacitive-coupling-flux-qubits\Modules\SQcircuit_extensions.py:901: ComplexWarning: Casting complex values to real discards the imaginary part
  H_eff[i, j] = (ψ_0[i].dag() * H * ψ_0[j]).data[0, 0]


In [19]:
P[-1,-1]/P_large[-1,-1]

-5.1088907883471295

In [20]:
ψ_0_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2, solver='numpy')[1][:,0])
ψ_1_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2, solver='numpy')[1][:,1])
ψ_0_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2, solver='numpy')[1][:,0])
ψ_1_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2, solver='numpy')[1][:,1])

ψ_0_composite = [qt.tensor(ψ_0_f, ψ_0_r), qt.tensor(ψ_1_f,ψ_0_r), qt.tensor(ψ_0_f,ψ_1_r), qt.tensor(ψ_1_f,ψ_1_r)]
E_0_composite = [ (ψ_0_i.dag() * H_0 * ψ_0_i).data[0,0]/2/np.pi/GHz for ψ_0_i in ψ_0_composite]

In [21]:
E_0, ψ_0_direct = sq_ext.diag(H_0  , n_eig=6, out='GHz', solver='numpy')
subspace_indices = sq_ext.find_close_indices(E_0_composite, E_0, tol=0.1)
E_0 = E_0[subspace_indices]
ψ_0_direct = ψ_0_direct[:,subspace_indices]

In [22]:
Q = np.zeros([4,4], dtype=np.complex128)
for i in range(4):
    for j in range(4):
        Q[i,j] = (ψ_0_composite[i].__array__().conj().T @ ψ_0_direct[:,j])[0]
        

In [23]:
np.round(Q,3)

array([[ 1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [-0.+0.j,  0.+0.j,  1.+0.j, -0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j,  1.+0.j]])

# SWT direct step by step

In [68]:
# H_eff_SWT = sq_ext.H_eff_SWT(H_0, H, n_eig=5) 
out='GHz'
real=False
remove_ground=False

In [69]:
ψ_0 = sq_ext.diag(H_0, 6, real=real, solver='numpy')[1]
E, ψ = sq_ext.diag(H, 6, real=False, solver='scipy', out=out)

In [70]:
ψ = ψ[:,subspace_indices]
ψ_0 = ψ_0[:,subspace_indices]
E = E[subspace_indices]

In [71]:
n_eig=4
Q = np.zeros((n_eig, n_eig), dtype=complex)
for i in range(n_eig):
    for j in range(n_eig):
        Q[i, j] = ψ_0[:,i].conj().T @ ψ[:,j]

In [72]:
np.round(Q,2)

array([[-0.71+0.71j,  0.  -0.j  , -0.  -0.j  , -0.  -0.02j],
       [ 0.  +0.j  ,  1.  +0.03j,  0.07-0.02j,  0.  +0.j  ],
       [ 0.  +0.j  , -0.07-0.j  ,  0.96-0.26j,  0.  +0.j  ],
       [-0.01+0.01j, -0.  -0.j  ,  0.  -0.j  ,  0.1 +0.98j]])

In [73]:


U, s, Vh = np.linalg.svd(Q)
A = U @ Vh

H_eff = A @ np.diag(E) @ A.T.conj()

In [74]:
sq_ext.decomposition_in_pauli_4x4(H_eff, print_pretty=True);

 9.879611414170412	*	 I ⨂I 
 -1.8425084880250704	*	 I ⨂σz 
 1.0812790766354108e-05	*	 σx ⨂σx 
 0.2048208512646924	*	 σy ⨂σy 
 -3.227244637743289	*	 σz ⨂I 
 0.0013862359205634167	*	 σz ⨂σz 


In [ ]:

Nf, Nr = sq_ext.sq_get_energy_indices_hamiltonian(H_qubit = H_0, H_fluxonium=H1, H_resonator=H2, n_eig=5)
subspace_indices = np.intersect1d(np.where(Nf<=1), np.where(Nr<=1)).tolist()

H_eff_SWT_subspace  = H_eff_SWT [subspace_indices][:, subspace_indices]
P = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_subspace, print_pretty=True)

# SWT comoposite step by step

In [93]:
ψ_0_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2, solver='numpy', real=True)[1][:,0])
ψ_1_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2, solver='numpy', real=True)[1][:,1])
ψ_0_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2, solver='numpy', real=True)[1][:,0])
ψ_1_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2, solver='numpy', real=True)[1][:,1])

ψ_0 = [qt.tensor(ψ_0_f, ψ_0_r), qt.tensor(ψ_1_f,ψ_0_r), qt.tensor(ψ_0_f,ψ_1_r),  qt.tensor(ψ_1_f,ψ_1_r)]

In [109]:
E_0_ψ_0 = [ (ψ_0_i.dag() * H_0 * ψ_0_i).data[0,0]/2/np.pi/GHz for ψ_0_i in ψ_0]
E_0, _ = sq_ext.diag(H_0  , n_eig=6, out='GHz', solver='Qutip', qObj=True)
E, ψ = sq_ext.diag(H  , n_eig=6, out='GHz', solver='Qutip', qObj=True)
subspace_indices = sq_ext.find_close_indices(E_0_ψ_0,E_0,tol=0.05)
ψ = ψ[subspace_indices]
E = E[subspace_indices]

In [110]:
# H_eff_SWT_large = sq_ext.H_eff_SWT_large(ψ_0, ψ, E)

In [111]:
n_eig = len(ψ_0)
Q = np.zeros((n_eig, n_eig), dtype=complex)
for i in range(n_eig):
    for j in range(n_eig):
        Q[i, j] = (ψ_0[i].dag() * ψ[j]).data[0,0]

U, s, Vh = np.linalg.svd(Q)
A = U @ Vh

H_eff = A @ np.diag(E) @ A.T.conj()

In [112]:
np.round(Q,2)

array([[-0.83+0.55j,  0.  +0.j  ,  0.  +0.j  ,  0.02+0.01j],
       [ 0.  +0.j  ,  0.75+0.66j, -0.02-0.07j,  0.  +0.j  ],
       [ 0.  +0.j  , -0.06-0.05j, -0.22-0.97j,  0.  +0.j  ],
       [ 0.02-0.01j,  0.  +0.j  ,  0.  +0.j  ,  0.93+0.34j]])

In [113]:

P_large = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_large, print_pretty=True)

 9.879611414170446	*	 I ⨂I 
 -1.8425084880250213	*	 I ⨂σz 
 0.20482085126469565	*	 σx ⨂σx 
 1.0812790769837433e-05	*	 σy ⨂σy 
 -3.227244637743288	*	 σz ⨂I 
 0.0013862359205289998	*	 σz ⨂σz 


In [114]:
sq_ext.decomposition_in_pauli_4x4(sq_ext.H_eff_SWT_large(ψ_0,ψ,E), print_pretty=True);

 9.879611414170494	*	 I ⨂I 
 -1.8425084880250775	*	 I ⨂σz 
 0.20482085126469565	*	 σx ⨂σx 
 1.0812790766673297e-05	*	 σy ⨂σy 
 -3.227244637743314	*	 σz ⨂I 
 0.0013862359205287778	*	 σz ⨂σz 


In [100]:
ψ_0

[Quantum object: dims = [[25, 15], [1, 1]], shape = (375, 1), type = ket
 Qobj data =
 [[ 9.85072988e-01]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 1.71435976e-01]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000